# CS470 Introduction to Artificial Intelligence
## Deep Learning Practice 
#### TA. Yechan Hwang
---

### Agenda for this practice
#### 1. Weather dataset
#### 2. Univariate model
#### 3. Normalizing the data
#### 4. Multivariate model
#### 5. Multi-step model
---
<br/>
<br/>
<br/>

## 6-2. Time series forecasting 
In this practice, we will use two approaches for time series forecasting.
- First, we will forecast a **univariate** time series
- Then we will forecast a **multivariate** time series.

In [1]:
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

#### The weather dataset
This practice uses a [`weather time series dataset`](https://www.bgc-jena.mpg.de/wetter/) (recorded by the Max-Planck-Institute for Biogeochemistry).

This dataset contains 14 different features such as air temperature, atmospheric pressure, and humidity. These were collected every 10 minutes, beginning in 2003. For efficiency, we will use only the data collected between 2009 and 2016. 

In [2]:
zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)

13574144/13568290 [==============================] - 1s 0us/step


<br/>
The data is in the form of CSV. If you are unfamiliar with handling the CSV data well in Tensorflow, check this tutorial:

[link](https://www.tensorflow.org/tutorials/load_data/pandas_dataframe)

In [ ]:
df = 

Let's take a glance at the data.

In [ ]:
df.head()

In [ ]:
df.tail()

As you can see above, an observation is recorded every 10 mintues from 01/01/2009. This means that, you will have 6 observations for a single hour. Similarly, a single day will contain 144 (6x24) observations.

For this prediction problem, you can specify any time period to predict. Let's say **you want to predict the temperature for 6 hours in the future**. And in order to make this prediction, let's say that **you decided to use 5 days of observations**. Then you can create a window containing the last 720 (5x144) observations to train the model and predict 6 hours in the future. Many such configurations are possible and you can choose best one after experiments with various input/output shapes.

<img src="images/example_model.png" alt="Drawing" style="width: 1100px;"/>

<br/>

Setting seed to ensure reproducibility.

In [6]:
tf.random.set_seed(13)

## Part 1: Forecast a univariate time series
First, we will train a model that learns **using only a single feature (temperature)** and use this model to make predictions for temperature values in the future.

Let's first extract only the temperature from the dataset.

In [ ]:
uni_data = 
uni_data.index = 

print(uni_data.head())
print(uni_data.shape)

<br/>

Let's observe how this data looks across time. By the `plot` function, we can visualize the temperature data.

First we will check global trend of temperature.

In [ ]:
uni_data.plot(subplots=True)

Also we can focus on specific period (from index 20000 to 21000).

In [ ]:
pd.Series(uni_data[20000:21000]).plot(subplots=True)

By using `Series.values`, we convert pandas data to numpy array.

In [ ]:
uni_data = uni_data.values
uni_data.shape

In both the following practices, we will use the first 300,000 rows of the data as training dataset and the remaining will be the validation dataset.

This amounts to ~3,000,000 minutes = ~2100 days worth of training data.

In [11]:
TRAIN_SPLIT = 300000
training_data = uni_data[:TRAIN_SPLIT]

It is important to **normalize features before training a neural network** since scales for each feature can be different significantly. A common way to do so is by subtracting the mean and dividing by the standard deviation of each feature.

And notice that the **mean and standard deviation should be computed using only the training data**.

In [ ]:
training_data_mean = 
training_data_std = 
print(training_data_mean,training_data_std)

Let's normalize the data and visualize them.

In [13]:
norm_uni_data = 

In [ ]:
pd.Series(norm_uni_data).plot(subplots=True)

### Univariate model

Let's now **create the data for the univariate model**. In this section, **the model will be given the last 20 recorded temperature observations** and learn to **predict the temperature of the next time step**.

<br/>

<img src="images/approach.png" alt="Drawing" style="width: 800px;"/>

<br/>

The helper function below returns the specific windows of time for the model to train on. The parameter `history_size` is the size of the window of past information. So our model will use the number of `history_size` information for prediction. The `target_size` defines how far in the future does the model need to learn to predict. In other words, if the `target_size` 0 is the above model, it will predict the temperature for time step 21 and it will predict the temparature for time step 22 if the `target_size` is 1.

In [15]:
def univariate_data(dataset, start_index, end_index, history_size, target_size):
    data = []
    labels = []
    
    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset)
    
    for i in range(start_index, end_index- target_size):
        indices = range(i-history_size, i)
        # Reshape data from (history_size,) to (history_size, 1)
        data.append(np.reshape(dataset[indices], (history_size, 1)))
        labels.append(dataset[i+target_size])
    return np.array(data), np.array(labels)

In [16]:
univariate_past_history = 20
univariate_future_target = 0

# Create training dataset with univariate feature
x_train_uni, y_train_uni = univariate_data(norm_uni_data, 0, TRAIN_SPLIT,
                                           univariate_past_history,
                                           univariate_future_target)

x_val_uni, y_val_uni = univariate_data(norm_uni_data, TRAIN_SPLIT, None,
                                       univariate_past_history,
                                       univariate_future_target)

In [ ]:
print(x_train_uni.shape)
print(x_val_uni.shape)

In [ ]:
norm_uni_data[:25]

In [ ]:
print ('Single window of past history')
print (x_train_uni[0])
print ('\n Target temperature to predict')
print (y_train_uni[0])

<br/><br/>

Now that the data has been created, let's take a look at a single example. The information given to the network is given in blue, and it must predict the value at the red cross.

In [20]:
def show_plot(plot_data, delta, title):
    labels = ['History', 'True Future', 'Model Prediction']
    marker = ['.-', 'rx', 'go']
    time_steps = range(-plot_data[0].shape[0],0)
    if delta:
        future = delta
    else:
        future = 0

    plt.title(title)
    for i, x in enumerate(plot_data):
        if i:
            plt.plot(future, plot_data[i], marker[i], markersize=10,
                label=labels[i])
        else:
            plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
    plt.legend()
    plt.xlim([time_steps[0], (future+5)*2])
    plt.xlabel('Time-Step')
    return plt

In [ ]:
show_plot([x_train_uni[0], y_train_uni[0]], 0, 'Sample Example')

#### Baseline
Before training our model, let's first **construct a simple baseline model**. Baseline model will looks at all the history observations and **predicts the next point to be the average them**.

In [22]:
def baseline(history):
    return np.mean(history)

In [ ]:
show_plot([x_train_uni[0], y_train_uni[0], baseline(x_train_uni[0])], 0,
           'Baseline Prediction Example')

Let's see if our model can beat this baseline using a recurrent neural network after training.

#### Recurrent neural network

In this practical, we will use a specialized RNN layer Long Short Tem Memory (LSTM).
Similar to RNN and GRU, LSTMs process a time series step-by-step, maintaining an internal state summarizing the information they've seen so far.

Let's now use `tf.data` to shuffle, make batch, and cache the dataset.

In [24]:
BATCH_SIZE = 256
BUFFER_SIZE = 10000

train_univariate = tf.data.Dataset.from_tensor_slices((x_train_uni, y_train_uni))
train_univariate = train_univariate.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_univariate = tf.data.Dataset.from_tensor_slices((x_val_uni, y_val_uni))
val_univariate = val_univariate.batch(BATCH_SIZE).repeat()

In [ ]:
for elem in train_univariate.take(1):
    print(elem)

We will make `simple_lstm_model` that has one LSTM with 8 units and one Dense layer with 1 unit. Also, since this problem is regression, we will use mean absolute error loss function.

In [26]:
simple_lstm_model = tf.keras.Sequential([
    

])

simple_lstm_model.compile(optimizer='adam', loss=    )

Let's make a sample prediction, to check the output of the model.

In [ ]:
for x, y in val_univariate.take(1):
    print(simple_lstm_model.predict(x).shape) # (BATCH_SIZE, Predicted_value)

Let's train the model now. Due to the large size of the dataset, in the interest of saving time, each epoch will only run for 200 steps, instead of the complete training data as normally done.

In [ ]:
EVALUATION_INTERVAL = 200
EPOCHS = 10

simple_lstm_model.fit(   __data__   , epochs=  ,
                      steps_per_epoch=  ,
                      validation_data=  , validation_steps=50)

#### Predict using the simple LSTM model
Now that you have trained your simple LSTM, let's try and make a few predictions.

In [ ]:
for x, y in val_univariate.take(3):
    plot = show_plot([x[0].numpy(), y[0].numpy(),
                      simple_lstm_model.predict(x)[0]], 0, 'Simple LSTM model')
    plot.show()

Now that you have seen the basics, let's move on to part two, where you will work with a multivariate time series.

## Part 2: Forecast a multivariate time series
The original dataset contains **fourteen features**. For simplicity, this part **considers only three of the original fourteen**. The features used are air temperature, atmospheric pressure, and air density.

To use more features, add their names to this list.

In [30]:
features_considered = ['p (mbar)', 'T (degC)', 'rho (g/m**3)']

In [ ]:
features = 
features.index = 
features.head()

Let's have a look at how each of these features vary across time.

In [ ]:
features.plot(subplots=True)

As mentioned, the **first step will be to normalize the dataset** using the mean and standard deviation of the training data.

In [33]:
dataset = 
data_mean = 
data_std =

In [34]:
dataset = 

#### Single step multivariate model
In a single step setup, **the model learns to predict a single point** in the future based on some history provided with multiple features.

The below function performs the same windowing task as above but now with many features.

In [35]:
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, single_step=False):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i, 1)
        data.append(dataset[indices])
        
        
        if single_step:
            labels.append(target[i+target_size])
        else:
            labels.append(target[i:i+target_size])

    return np.array(data), np.array(labels)

In this practice, the network will be given past 20 observations, but now each observations have three different features. Therefore, the amount of information the model can refer when making predictions has increased. But at the same time, the amount of information the model has to learn during training has also increased.

Similar to what we did above, our model will be given the last 20 recorded temperature observations and learn to predict the temperature of the next time step.

In [36]:
past_history = 20
future_target = 0

x_train_single, y_train_single = multivariate_data(

)

x_val_single, y_val_single = multivariate_data(

)

Let's look at a single data-point.

In [ ]:
# 20 observations with 3 features 
print ('Single window of past history : {}'.format(x_train_single[0].shape))

In [38]:
train_data_single = tf.data.Dataset.from_tensor_slices((x_train_single, y_train_single))
train_data_single = train_data_single.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data_single = tf.data.Dataset.from_tensor_slices((x_val_single, y_val_single))
val_data_single = val_data_single.batch(BATCH_SIZE).repeat()

Now we will make `single_step_model` that has one LSTM with 32 units and one Dense layer with 1 unit.

In [39]:
single_step_model = tf.keras.models.Sequential([


])

# mae : mean absolute error
single_step_model.compile(optimizer='rmsprop', loss='mae')

Now we will train our model and plot the training and validation loss on the graph.

In [ ]:
EVALUATION_INTERVAL = 200
EPOCHS = 10

single_step_history = single_step_model.fit(


)

In [41]:
def plot_train_history(history, title):
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(loss))

    plt.figure()

    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title(title)
    plt.legend()

    plt.show()

In [ ]:
plot_train_history(single_step_history,
                   'Single Step Training and validation loss')

#### Predict a single step future
Now that the model is trained, let's make a few sample predictions.

In [ ]:
for x, y in val_data_single.take(3):
    plot = show_plot([x[0][:, 1].numpy(), y[0].numpy(),
                    single_step_model.predict(x)[0]], 0,
                   'Single Step Prediction')
    
    plot.show()

When we compare our model's prediction with the univariate model, it seems that our model makes predictions better.

##### Will increasing the number of features always increase performance?

It depends on your dataset. While some features have very useful information for your model's prediction, some features may not. 

Although there are various methods for choosing which features to use (e.g. using the feature's statistics, PCA, embedding, etc.), we will not cover them in this practice.

<br/><br/>

#### Multi-step prediction model

In a multi-step prediction model, given a past history, the model needs to learn to **predict a range of future values**. Thus, unlike a single step model, where only a single future point is predicted, **a multi-step model predict a sequence of the future**.

For the multi-step model, the training data again consists of past 20 observations that have three different features. However, now the model needs to learn to predict the temperature **for the next 3 time steps**. For this task, the dataset needs to be preprocessed again so that each target object has 3 labels.


<br/><br/>
<img src="images/multi.png" alt="Drawing" style="width: 800px;"/>

In [44]:
past_history = 20
future_target = 3

x_train_multi, y_train_multi = multivariate_data(dataset, dataset[:, 1], 0,
                                                 TRAIN_SPLIT, past_history,
                                                 future_target, single_step=False)
x_val_multi, y_val_multi = multivariate_data(dataset, dataset[:, 1],
                                             TRAIN_SPLIT, None, past_history,
                                             future_target, single_step=False)

Let's check out a sample data-point.

In [ ]:
print ('Single window of past history : {}'.format(x_train_multi[0].shape))
print ('\nTarget temperature to predict : {}'.format(y_train_multi[0].shape))

In [46]:
train_data_multi = tf.data.Dataset.from_tensor_slices((x_train_multi, y_train_multi))
train_data_multi = train_data_multi.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data_multi = tf.data.Dataset.from_tensor_slices((x_val_multi, y_val_multi))
val_data_multi = val_data_multi.batch(BATCH_SIZE).repeat()

`multi_step_plot` is a helper function that visualizes many True/Predicted points on the plot.

In [47]:
def multi_step_plot(history, true_future, prediction):
    plt.figure(figsize=(12, 6))
    num_in = range(-len(history),0)
    num_out = len(true_future)

    plt.plot(num_in, np.array(history[:, 1]), label='History')
    plt.plot(np.arange(num_out), np.array(true_future), 'bo',
           label='True Future')
    if prediction.any():
        plt.plot(np.arange(num_out), np.array(prediction), 'ro',
                 label='Predicted Future')
    plt.legend(loc='upper left')
    plt.show()

In [ ]:
for x, y in train_data_multi.take(1):
    multi_step_plot(x[0], y[0], np.array([0]))

Since the task here is a bit more complicated than the previous task, **the model now consists of two LSTM layers**. 

`multi_step_model` will have
- LSTM with 32 units as input
- LSTM with 16 units
- Dense layer with 3 units

Note that the last Dense layer has 3 nodes in order to predict 3 target values.

In [49]:
multi_step_model = tf.keras.models.Sequential([


])

multi_step_model.compile(optimizer=tf.keras.optimizers.RMSprop(clipvalue=1.0), loss='mae')

In [ ]:
EVALUATION_INTERVAL = 200
EPOCHS = 10

multi_step_history = multi_step_model.fit(train_data_multi, epochs=EPOCHS,
                                          steps_per_epoch=EVALUATION_INTERVAL,
                                          validation_data=val_data_multi,
                                          validation_steps=50)

In [ ]:
plot_train_history(multi_step_history, 'Multi-Step Training and validation loss')

#### Predict a multi-step future
Let's now have a look at how well your network has learnt to predict the future. 

In [ ]:
for x, y in val_data_multi.take(3):
    multi_step_plot(x[0], y[0], multi_step_model.predict(x)[0])

You can experiment with various model configurations and select best one. Your options are as follows:

- Increasae/decrease the context length
- Change the number of labels the model will predict
- Change the number of parameters of LSTM
- Change features to include in learning